# Esempio di Agente Prenotazione Hotel e Voli

Questa soluzione ti aiuterà a prenotare biglietti aerei e hotel. Lo scenario è un viaggio da Londra Heathrow LHR il 20 febbraio 2024 a New York JFK con ritorno il 27 febbraio 2025, volando in classe economica solo con British Airways. Voglio soggiornare in un hotel Hilton a New York, per favore forniscimi i costi per il volo e l'hotel.


# Inizializzare il Servizio Azure AI Agent e ottenere informazioni di configurazione da **.env**

### **.env** 

Crea un file .env 

**.env** contiene la stringa di connessione del Servizio Azure AI Agent, il modello utilizzato da AOAI e il corrispondente servizio API di ricerca di Google, ENDPOINT, ecc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nome del Deployment del Modello del Servizio Azure AI Agent"

[**NOTE**] Avrai bisogno di un modello con un limite di 100.000 Token al minuto e un limite di 600 richieste al minuto.

  Puoi ottenere il modello in Azure AI Foundry - Modello ed Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Stringa di Connessione del Progetto del Servizio Azure AI Agent"

  Puoi ottenere la stringa di connessione del progetto nella panoramica del tuo progetto nella schermata del portale AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "La tua chiave API di ricerca SERPAPI"
- **SERPAPI_SEARCH_ENDPOINT** = "Il tuo endpoint di ricerca SERPAPI"

Per ottenere il Nome del Deployment del Modello e la Stringa di Connessione del Progetto del Servizio Azure AI Agent, è necessario creare il Servizio Azure AI Agent. Si consiglia di utilizzare [questo template](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) per crearlo direttamente （***Nota:*** Il Servizio Azure AI Agent è attualmente disponibile in regioni limitate. Si consiglia di fare riferimento a [questo link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) per impostare la regione)

L'Agent deve accedere a SERPAPI. Si consiglia di registrarsi utilizzando [questo link](https://serpapi.com/searches). Dopo la registrazione, puoi ottenere una chiave API e un ENDPOINT unici.


# Configurazione

Per eseguire questo notebook, assicurati di aver installato le librerie necessarie eseguendo `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

La tua versione di Semantic Kernel dovrebbe essere almeno 1.27.2.


Carica le impostazioni e le risorse del tuo file .env, assicurati di aver aggiunto le tue chiavi e impostazioni e di aver creato un file .env locale.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Accedi ad Azure

Ora devi accedere ad Azure. Apri un terminale ed esegui il seguente comando:

```bash
az login
```

Questo comando ti chiederà di inserire le tue credenziali Azure, permettendo al servizio Azure AI Agent di funzionare correttamente.


# Spiegazione:
Questa è una variabile che memorizza la chiave API per accedere a un servizio API SERP (Search Engine Results Page). Una chiave API è un identificatore unico utilizzato per autenticare le richieste associate al tuo account.

Scopo: Lo scopo di questa riga è memorizzare la chiave API in una variabile, in modo che possa essere utilizzata per autenticare le richieste al servizio API SERP. La chiave API è necessaria per accedere al servizio e effettuare ricerche.
Come ottenere una chiave API SERP: Per ottenere una chiave API SERP, segui questi passaggi generali su https://serpapi.com (i passaggi esatti possono variare a seconda del servizio API SERP specifico che stai utilizzando):

Scegli un servizio API SERP: Esistono diversi servizi API SERP disponibili, come SerpAPI, Google Custom Search JSON API e altri. Scegli quello che meglio si adatta alle tue esigenze.

Registrati per un account: Vai al sito web del servizio API SERP scelto e registrati per un account. Potrebbe essere necessario fornire alcune informazioni di base e verificare il tuo indirizzo email.

Crea una chiave API: Dopo esserti registrato, accedi al tuo account e vai alla sezione API o al dashboard. Cerca un'opzione per creare o generare una nuova chiave API.
Copia la chiave API nel tuo file .env.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Spiegazione:
BASE_URL: Questa è una variabile che memorizza l'URL di base per l'endpoint dell'API SERP. Il nome della variabile BASE_URL è una convenzione utilizzata per indicare che questo URL è il punto di partenza per effettuare richieste API.
'https://serpapi.com/search':

Questo è il valore effettivo della stringa URL assegnata alla variabile BASE_URL. Rappresenta l'endpoint per eseguire query di ricerca utilizzando l'API SERP.

# Scopo:
Lo scopo di questa riga è definire una costante che contiene l'URL di base per l'API SERP. Questo URL sarà utilizzato come punto di partenza per costruire richieste API per eseguire operazioni di ricerca.

# Utilizzo:
Definendo l'URL di base in una variabile, puoi riutilizzarlo facilmente nel tuo codice ogni volta che devi effettuare richieste all'API SERP. Questo rende il tuo codice più manutenibile e riduce il rischio di errori derivanti dall'inserimento manuale dell'URL in più punti. L'esempio attuale è https://serpapi.com/search?engine=bing che utilizza l'API di ricerca Bing. È possibile selezionare API diverse su https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Spiegazione:

Questo è il luogo in cui si trova il codice del tuo plugin.

Definizione della Classe: `class BookingPlugin`: Definisce una classe chiamata BookingPlugin che contiene metodi per prenotare hotel e voli.

Metodo di Prenotazione Hotel:

- `@kernel_function(description="booking hotel")`: Un decoratore che descrive la funzione come una funzione kernel per la prenotazione di hotel.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Definisce un metodo per prenotare hotel con parametri annotati e tipo di ritorno.

Il metodo costruisce un dizionario di parametri per la richiesta di prenotazione dell'hotel e invia una richiesta GET all'API SERP. Controlla lo stato della risposta e restituisce le proprietà dell'hotel se la richiesta ha successo, oppure None se la richiesta non è riuscita.

Metodo di Prenotazione Voli:

- `@kernel_function(description="booking flight")`: Un decoratore che descrive la funzione come una funzione kernel per la prenotazione di voli.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Definisce un metodo per prenotare voli con parametri annotati e tipo di ritorno.

Il metodo costruisce dizionari di parametri per le richieste di volo di andata e ritorno e invia richieste GET all'API SERP. Controlla lo stato della risposta e aggiunge le informazioni sui voli alla stringa di risultato se la richiesta ha successo, oppure stampa un messaggio di errore se la richiesta non è riuscita. Il metodo restituisce la stringa di risultato contenente le informazioni sui voli.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Spiegazione:
Importazione dei moduli: Importa i moduli necessari per le credenziali di Azure, l'agente AI, il contenuto dei messaggi di chat, il ruolo dell'autore e il decoratore delle funzioni del kernel.

Gestore di contesto asincrono: `async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`: Configura un gestore di contesto asincrono per gestire le credenziali di Azure e creare un client per l'agente AI.

Nome e istruzioni dell'agente:
- `AGENT_NAME = "BookingAgent"`: Definisce il nome dell'agente.
- `AGENT_INSTRUCTIONS = """..."""`: Fornisce istruzioni dettagliate all'agente su come gestire le richieste di prenotazione.

Creazione della definizione dell'agente: `agent_definition = await client.agents.create_agent(...)`: Crea una definizione dell'agente con il modello, il nome e le istruzioni specificate.

Creazione dell'agente AzureAI: `agent = AzureAIAgent(...)`: Crea un agente AzureAI utilizzando il client, la definizione dell'agente e il plugin definito.

Creazione del thread: `thread: AzureAIAgentThread | None = None`: Crea un thread per l'agente. Non è necessario creare prima un thread - se viene fornito il valore `None`, un nuovo thread verrà creato durante la prima invocazione e restituito come parte della risposta.

Input dell'utente: `user_inputs = ["..."]`: Definisce un elenco di input dell'utente che l'agente deve elaborare.

Nel blocco `finally`, elimina il thread e l'agente per liberare le risorse.


# Autenticazione

La classe `DefaultAzureCredential` fa parte dell'SDK di Azure per Python. Fornisce un metodo predefinito per autenticarsi con i servizi di Azure. Tenta di autenticarsi utilizzando diversi metodi in un ordine specifico, come variabili d'ambiente, identità gestita e credenziali di Azure CLI.

Operazioni asincrone: Il modulo aio indica che la classe DefaultAzureCredential supporta operazioni asincrone. Questo significa che puoi utilizzarla con asyncio per eseguire richieste di autenticazione non bloccanti.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa deve essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale eseguita da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
